In [1]:
import torch
import torch.utils.data as data
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
import PIL.Image as Image
import numpy as np

In [5]:
train_data = datasets.MNIST(
    root="data/DataSet/",
    download=True,
    train=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            mean=(0.5),
            std=(0.5)
        )
    ])
)

train_loader = data.DataLoader(
    dataset=train_data,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

In [ ]:
class Generator(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        # 1x1x1 -> 
        self.layer1 = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=in_channels,    
                out_channels=
            )
        )